# Introduction

I have no idea if this is going to work, but I wanted to see how an RNN with LSTM units would do with this type of data. 

In [46]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [54]:
batchSize = 24
lstmUnits = 24
numClasses = 2
iterations = 100000
maxSeqLength = 17
numDimensions = 1
numFeatures = 17

In [55]:
from sklearn.model_selection import train_test_split
from numpy import newaxis
xTrain = np.load('PrecomputedMatrices/xTrain.npy')
yTrain = np.load('PrecomputedMatrices/yTrain.npy')
xTrain = xTrain[:, :, newaxis]
X_train, X_test, y_train, y_test = train_test_split(xTrain, yTrain, test_size = 0.20)

print (X_train.shape)
print (y_train.shape)

(92090, 17, 1)
(92090,)


# RNN Model 

In [56]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.float32, [batchSize, maxSeqLength, numDimensions])

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.85)
value, _ = tf.nn.dynamic_rnn(lstmCell, input_data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

# Training

In [57]:
import datetime

sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [58]:
from random import randint
def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    num = randint(0,X_train.shape[0] - batchSize)
    l = y_train[num:num+batchSize]
    for num in l:
        if (num == 0):
            labels.append([1, 0])
        else:
            labels.append([0, 1])
    arr = X_train[num:num+batchSize]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    num = randint(0,X_test.shape[0] - batchSize)
    l = y_test[num:num+batchSize]
    for num in l:
        if (num == 0):
            labels.append([1, 0])
        else:
            labels.append([0, 1])
    arr = X_test[num:num+batchSize]
    return arr, labels

In [59]:
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
    #Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

    #Save the network every 10,000 training iterations
#    if (i % 10000 == 0 and i != 0):
#        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
#        print("saved to %s" % save_path)
writer.close()

/Users/adit/anaconda/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Didn't work :(

![](Data/Accuracy.png)